In [1]:
import uuid
class Account:
    def __init__(self, acc_id,cust_id, bal=0.0):
        self._account_number=acc_id
        self._account_holder_id=cust_id
        self._balance=bal
    def deposit(self,amt):
        if amt<=0: return False
        self._balance+=amt
        return True
    def withdraw(self, amt):
        return False
    def display_details(self):
        return f"Acc No: {self._account_number}, Bal: {self._balance:.2f}"

class SavingsAccount(Account):
    def __init__(self,acc_id,cust_id,bal=0.0,rate=0.01):
        super().__init__(acc_id, cust_id, bal)
        self._interest_rate=rate
    def withdraw(self, amt):
        if 0<amt<=self._balance:
            self._balance-=amt
            return True
        return False
    def apply_interest(self):
        interest=self._balance*(self._interest_rate/100)
        self._balance+=interest
    def display_details(self):
        return super().display_details() + f", Interest: {self._interest_rate}%"

class CheckingAccount(Account):
    def __init__(self,acc_id,cust_id,bal=0.0,limit=0.0):
        super().__init__(acc_id, cust_id,bal)
        self._overdraft_limit =limit
    def withdraw(self, amt):
        if self._balance - amt >= -self._overdraft_limit:
            self._balance -= amt
            return True
        return False
    def display_details(self):
        return super().display_details() + f", Overdraft: {self._overdraft_limit}"

class Customer:
    def __init__(self, cid, name, pwd, addr):
        self._id = cid
        self._name = name
        self._password = pwd
        self._address = addr
        self._accounts = []
    def add_account(self, acc_id):
        self._accounts.append(acc_id)
    def display(self):
        return f"ID: {self._id}, Name: {self._name}, Addr: {self._address}, Accounts: {len(self._accounts)}"

class Bank:
    def __init__(self):
        self.customers = {}
        self.accounts = {}
        self.load()
    def save(self):
        with open("bank_data.txt", "w") as f:
            for cid, c in self.customers.items():
                f.write(f"C|{cid}|{c._name}|{c._password}|{c._address}|{','.join(c._accounts)}\n")
            for aid, a in self.accounts.items():
                if isinstance(a, SavingsAccount):
                    f.write(f"S|{aid}|{a._account_holder_id}|{a._balance}|{a._interest_rate}\n")
                elif isinstance(a, CheckingAccount):
                    f.write(f"K|{aid}|{a._account_holder_id}|{a._balance}|{a._overdraft_limit}\n")
    def load(self):
        try:
            with open("bank_data.txt") as f:
                for line in f:
                    parts = line.strip().split("|")
                    if parts[0] == "C":
                        cid, name, pwd, addr, accs = parts[1:]
                        cust = Customer(cid, name, pwd, addr)
                        cust._accounts = accs.split(",") if accs else []
                        self.customers[cid] = cust
                    elif parts[0] == "S":
                        aid, cid, bal, rate = parts[1:]
                        self.accounts[aid] = SavingsAccount(aid, cid, float(bal), float(rate))
                    elif parts[0] == "K":
                        aid, cid, bal, limit = parts[1:]
                        self.accounts[aid] = CheckingAccount(aid, cid, float(bal), float(limit))
        except:
            pass
    def login(self, cid, pwd):
        return self.customers.get(cid) if self.customers.get(cid) and self.customers[cid]._password == pwd else None
    def add_customer(self, name, pwd, addr):
        cid = str(uuid.uuid4())[:8]
        self.customers[cid] = Customer(cid, name, pwd, addr)
        self.save()
        return cid
    def create_account(self, cid, acc_type, amt=0.0, extra=0.01):
        aid = str(uuid.uuid4())[:8]
        if acc_type.lower().startswith('s'): 
            self.accounts[aid] = SavingsAccount(aid, cid, amt, extra)
        else:
            self.accounts[aid] = CheckingAccount(aid, cid, amt, extra)
        self.customers[cid].add_account(aid)
        self.save()
        return aid
    def deposit(self, aid, amt):
        ok = self.accounts[aid].deposit(amt)
        if ok: self.save()
        return ok
    def withdraw(self, aid, amt):
        ok = self.accounts[aid].withdraw(amt)
        if ok: self.save()
        return ok
    def transfer(self, from_aid, to_aid, amt):
        if from_aid not in self.accounts or to_aid not in self.accounts:
            print("Invalid account ID(s).")
            return False
        print(f"Before transfer:")
        print(f"From ({from_aid}) balance: {self.accounts[from_aid]._balance}")
        print(f"To   ({to_aid}) balance: {self.accounts[to_aid]._balance}")
        if self.withdraw(from_aid, amt):
            if self.deposit(to_aid, amt):
                self.save()
                print(f"After transfer:")
                print(f"From ({from_aid}) balance: {self.accounts[from_aid]._balance}")
                print(f"To   ({to_aid}) balance: {self.accounts[to_aid]._balance}")
                return True
            else:
                self.deposit(from_aid, amt)
        return False
    def apply_interest_all(self):
        for acc in self.accounts.values():
            if isinstance(acc, SavingsAccount):
                acc.apply_interest()
        self.save()
    def get_accounts(self, cid):
        return [self.accounts[aid] for aid in self.customers[cid]._accounts]
def main():
    bank = Bank()
    while True:
        print("WELCOME TO THE WORLD BANK!")
        print("\n1.Signup \n2.Login \n3.Exit")
        ch = input("Choice: ")
        if ch == '1':
            cid = bank.add_customer(input("Name: "), input("Password: "), input("Address: "))
            print("Your ID:", cid)
        elif ch == '2':
            cid = input("ID: ")
            pwd = input("Password: ")
            user = bank.login(cid, pwd)
            if user:
                print("Welcome,", user._name)
                while True:
                    print("\n1.Create Account \n2.Deposit \n3.Withdraw \n4.Transfer \n5.My Account \n6.Interest \n7.Logout")
                    c = input(">> ")
                    if c == '1':
                        t = input("Type (Savings/Checking): ")
                        a = float(input("Amount: "))
                        x = float(input("Rate/Limit: "))
                        aid = bank.create_account(cid, t, a, x)
                        print("Account created:", aid)
                    elif c == '2':
                        aid = input("Account ID: ")
                        amt = float(input("Amount: "))
                        print("Success" if bank.deposit(aid, amt) else "Fail")
                    elif c == '3':
                        aid = input("Account ID: ")
                        amt = float(input("Amount: "))
                        print("Success" if bank.withdraw(aid, amt) else "Fail")
                    elif c == '4':
                        f = input("From Account: ")
                        t = input("To Account: ")
                        amt = float(input("Amount: "))
                        print("Success" if bank.transfer(f, t, amt) else "Fail")
                    elif c == '5':
                        for a in bank.get_accounts(cid):
                            print(a.display_details())
                    elif c == '6':
                        bank.apply_interest_all()
                        print("Applied interest.")
                        for a in bank.get_accounts(cid):
                            print(a.display_details())
                    elif c == '7':
                        break
            else:
                print("Login failed.")
        else:
            break
main()

WELCOME TO THE WORLD BANK!

1.Signup 
2.Login 
3.Exit


Choice:  1
Name:  Krishna Singh
Password:  pass1
Address:  Lucknow


Your ID: 4d854cdd
WELCOME TO THE WORLD BANK!

1.Signup 
2.Login 
3.Exit


Choice:  1
Name:  Harsh Kumar Singh
Password:  pass2
Address:  Varanasi


Your ID: e5c6e464
WELCOME TO THE WORLD BANK!

1.Signup 
2.Login 
3.Exit


...

>>  4
From Account:  54755372
To Account:  e9f59eff
Amount:  1000


Before transfer:
From (54755372) balance: 10000.0
To   (e9f59eff) balance: 6100.0
After transfer:
From (54755372) balance: 9000.0
To   (e9f59eff) balance: 7100.0
Success

1.Create Account 
2.Deposit 
3.Withdraw 
4.Transfer 
5.My Account 
6.Interest 
7.Logout


>>  7


WELCOME TO THE WORLD BANK!

1.Signup 
2.Login 
3.Exit


Choice:  3
